<div align="center">
    <img src="images/um_logo.png" alt="image">
</div>

# Computación II


# Introducción a FIFOs en Linux

## ¿Qué es un FIFO?

FIFO, acrónimo de "First In, First Out", es un tipo de archivo especial en sistemas operativos Unix/Linux que se utiliza para la comunicación entre procesos. Actúa como un conducto, permitiendo que los datos fluyan de un proceso a otro de forma unidireccional. Los FIFOs también se conocen como "named pipes" porque, a diferencia de los pipes anónimos, tienen un nombre en el sistema de archivos.

FIFOs son útiles para la comunicación entre procesos (IPC) donde los procesos de ejecución independiente necesitan intercambiar datos de manera segura y ordenada.

## Creación y Uso de FIFOs

### Creación de un FIFO

Para crear un FIFO, se utiliza el comando `mkfifo` en la terminal. Por ejemplo:

```bash
mkfifo my_fifo
```

Esto creará un FIFO llamado my_fifo en el directorio actual.

### Escritura y Lectura en un FIFO
Los FIFOs se utilizan con operaciones estándar de lectura y escritura. Un proceso escribe datos en el FIFO utilizando la syscall (llamada al sistema) `write()` y otro proceso puede leer estos datos utilizando `read()`.

Ejemplo de Escritura:

In [1]:
# Escritura en un FIFO usando Python
import os

fifo_path = 'my_fifo'
with open(fifo_path, 'w') as fifo:
    fifo.write('Hola desde el escritor!')


Ejemplo de Lectura:

In [2]:
# Lectura de un FIFO usando Python
import os

fifo_path = 'my_fifo'
with open(fifo_path, 'r') as fifo:
    message = fifo.read()
    print(f'Mensaje recibido: {message}')


FileNotFoundError: [Errno 2] No such file or directory: 'my_fifo'

## Consideraciones Importantes
- __Bloqueo:__ Los FIFOs bloquean la ejecución hasta que ambos extremos están conectados. El proceso de lectura se bloqueará hasta que haya datos para leer, y el proceso de escritura se bloqueará si el buffer está lleno.
- __Sincronización:__ Los FIFOs no son adecuados para situaciones donde se requiere acceso concurrente por múltiples procesos sin un esquema de control.
- __Seguridad:__ Es importante gestionar los permisos de acceso al FIFO para evitar problemas de seguridad.

## Casos de Uso Comunes
- __Intercomunicación entre servicios:__ Por ejemplo, un servidor web puede comunicarse con un script de backend a través de un FIFO.
- __Programación de flujos de trabajo:__ En un pipeline de procesamiento de datos, varios procesos pueden pasar datos de uno a otro secuencialmente usando FIFOs.

# Códigos de ejemplo

In [4]:
import os
import time


pipe_name = '/tmp/pipe_test'


def child():
    pipeout = os.open(pipe_name, os.O_WRONLY)
    counter = 0

    while True:
        os.write(pipeout, b'Number %03d\n' % counter)         
        counter = (counter+1) % 5
        time.sleep(1)


def child_v2():
    pipeout = open(pipe_name, 'w')
    counter = 0

    while True:
        pipeout.write('Number %03d\n' % counter)
        counter = (counter+1) % 5
        pipeout.flush()
        time.sleep(1)


def parent():
    pipein = open(pipe_name, 'r')

    while True:
        line = pipein.readline()[:-1]
        print('Parent %d got "%s" al %s' % (os.getpid(), line, time.time()))


if not os.path.exists(pipe_name):
    os.mkfifo(pipe_name)


pid = os.fork()

if pid != 0:
    parent()
else:
    child_v2()

Parent 8447 got "Number 000" al 1713229028.0453935
Parent 8447 got "Number 001" al 1713229029.046153
Parent 8447 got "Number 002" al 1713229030.047356
Parent 8447 got "Number 003" al 1713229031.048404
Parent 8447 got "Number 004" al 1713229032.0495973
Parent 8447 got "Number 000" al 1713229033.0511074
Parent 8447 got "Number 001" al 1713229034.052306
Parent 8447 got "Number 002" al 1713229035.0535147
Parent 8447 got "Number 003" al 1713229036.0546184
Parent 8447 got "Number 004" al 1713229037.055826
Parent 8447 got "Number 000" al 1713229038.056357
Parent 8447 got "Number 001" al 1713229039.057531
Parent 8447 got "Number 002" al 1713229040.058767
Parent 8447 got "Number 003" al 1713229041.0598705
Parent 8447 got "Number 004" al 1713229042.0613208
Parent 8447 got "Number 000" al 1713229043.0624855
Parent 8447 got "Number 001" al 1713229044.063935
Parent 8447 got "Number 002" al 1713229045.0650396
Parent 8447 got "Number 003" al 1713229046.0665557
Parent 8447 got "Number 004" al 17132290

KeyboardInterrupt: 

El código proporcionado implementa una comunicación de tipo FIFO (First-In-First-Out) utilizando tuberías en un entorno Linux. Utiliza el módulo `os` para interactuar con el sistema operativo y el módulo `time` para manejar temporizaciones.

### Estructura del Código:
- **Creación de la tubería**: Se verifica si ya existe una tubería en la ruta `/tmp/pipe_test`. Si no existe, se crea una nueva utilizando `os.mkfifo`.

- **Forking del proceso**: Se utiliza `os.fork()` para crear un proceso hijo. Dependiendo del valor devuelto (`pid`), el script ejecutará el comportamiento del padre o del hijo.

### Comportamiento del Proceso Hijo:
- **Versión 1 (`child`)**: Abre la tubería en modo escritura, escribe una cadena de texto que incluye un contador que se reinicia después de alcanzar 5, y espera un segundo entre cada escritura.
- **Versión 2 (`child_v2`)**: Similar a la versión 1 pero utilizando la sintaxis de manejo de archivos de alto nivel de Python (`open()` en lugar de `os.open()`), e incorpora `flush()` para asegurar que el contenido se escribe en la tubería sin retrasos.

### Comportamiento del Proceso Padre (`parent`):
- **Lectura de la tubería**: El proceso padre abre la tubería en modo lectura, lee las líneas enviadas por el hijo y las imprime con un sello temporal.

Este enfoque permite una comunicación entre procesos en la misma máquina de manera sincronizada y en tiempo real, utilizando el concepto FIFO donde el primer dato en ser enviado es el primero en ser recibido y procesado.

----------------------------------------------------------------------------------


In [ ]:
import os
import time

fifo_path = '/tmp/my_fifo'

# Crear el FIFO si no existe
if not os.path.exists(fifo_path):
    os.mkfifo(fifo_path)

print("Opening FIFO...")
with open(fifo_path, 'w') as fifo:
    print("FIFO opened for writing.")
    for i in range(10):
        message = f'Message {i}'
        print(f'Sending: {message}')
        fifo.write(message + '\n')
        fifo.flush()  # Asegura que el mensaje se escriba inmediatamente
        time.sleep(1)  # Espera un segundo entre mensajes

print("Done writing messages.")

In [ ]:
import os

fifo_path = '/tmp/my_fifo'

print("Opening FIFO...")
with open(fifo_path, 'r') as fifo:
    print("FIFO opened for reading.")
    while True:
        message = fifo.readline().strip()
        if message == "":
            print("No more messages. Exiting.")
            break
        print(f'Received: {message}')

Los programas proporcionados son dos scripts simples en Python que utilizan FIFO para comunicarse entre dos procesos en un sistema Linux. El código gestiona la creación, escritura y lectura de una tubería nombrada.

### Escritura en la FIFO:
- **Creación de la FIFO**: El script verifica si existe un archivo en la ruta `/tmp/my_fifo`. Si no existe, se crea usando `os.mkfifo`, que es específico para crear tuberías nombradas en sistemas Unix-like.
- **Manejo de la tubería**: Abre la tubería en modo escritura. Envía diez mensajes, uno por uno, asegurando que cada mensaje se escriba inmediatamente en la FIFO mediante `flush()`, y pausando un segundo entre cada envío con `time.sleep()`. Esto simula un flujo de datos a intervalos regulares.

### Lectura desde la FIFO:
- **Apertura y lectura de la FIFO**: Se abre la misma tubería en modo lectura. El script lee continuamente de la tubería hasta que no hay más datos (es decir, `readline()` devuelve una cadena vacía). Cada mensaje recibido se imprime en la consola.

### Consideraciones:
- **Sincronización**: Los scripts dependen de que ambos procesos estén corriendo simultáneamente para la correcta sincronización entre la escritura y la lectura, pues la FIFO no almacena datos entre ejecuciones.
- **Uso práctico**: Este tipo de comunicación es útil en aplicaciones donde los procesos deben comunicarse de manera secuencial y en tiempo real, como en sistemas de procesamiento de comandos o transmisión de datos sensoriales en tiempo real.
- ----------------------------------------------------------------
